In [1]:
import pandas as pd
import requests
import psycopg2
from psycopg2 import sql


### Extract customer id and email from the table customer

In [2]:
db_params = {
            "host":'localhost',
            "database":'postgres',
            "user":'postgres',
            "password":'!q2W3e4r'
            }

connection = psycopg2.connect(**db_params)

# the actual name of your PostgreSQL table
table_name = 'customers'
column_name1 = 'id'
colum_name2 = 'contact_email'

# Construct the SQL query to select all columns from the table
query = f"SELECT {column_name1},{colum_name2} FROM {table_name}"

# Use pandas to read the data into a DataFrame
df = pd.read_sql_query(query, connection)

# Display the DataFrame
print(df)

# Close the database connection
connection.close()

df_backup = df.copy()


           id                      contact_email
0     1023597                casparcelal@rust.de
1     1058556         zoltanweitzel@bolander.com
2     1011801              antonios62@scheibe.de
3     1062069             rudolphjan@steckel.com
4     1071647              sarinaernst@plath.org
...       ...                                ...
3696  1021042     nicole66@caldwell-crawford.biz
3697  1057037    christopher90@murphy-morrow.com
3698  1090115  racheldominguez@stevens-evans.net
3699  1025342                 mannlisa@drake.com
3700  1036675              jennifer20@palmer.biz

[3701 rows x 2 columns]


### Extract the domain name and get the Industry values from the API

In [3]:
def extractDomain(x):
    return x.strip().split('@')[1]

email = df['contact_email'].apply(extractDomain)
email.drop_duplicates(inplace=True)
email = email.reset_index(drop=True)

df = pd.DataFrame(email)
df['industry'] = None

In [8]:
url = "https://api.apollo.io/v1/organizations/enrich"

headers = {
    'Cache-Control': 'no-cache',
    'Content-Type': 'application/json'
}


for i in range(0,len(df)):
    domain = df.iloc[i,0] 
    querystring = {
        "api_key": "ehMfh3qDOoYl174_gXvXeg",
        "domain": domain
    }
    try:
        response = requests.request("GET", url, headers=headers, params=querystring)
        print(i)
        if bool(response.json()):
            industry = response.json()
            df.iloc[i,1] = industry["organization"]["industry"]
    except IOError as e:
    # Handle a more general exception (if any other unexpected exception occurs)
        print(f"Unexpected error: {e}")
    except Exception as e:
    # Print the exact error message
        print(f"An error occurred: {e}")


print(df)

0
An error occurred: 'organization'
1
An error occurred: 'organization'
2
An error occurred: 'organization'
3
An error occurred: 'organization'
4
An error occurred: 'organization'
5
An error occurred: 'organization'
6
An error occurred: 'organization'
7
An error occurred: 'organization'
8
An error occurred: 'organization'
9
An error occurred: 'organization'
10
An error occurred: 'organization'
11
An error occurred: 'organization'
12
An error occurred: 'organization'
13
An error occurred: 'organization'


KeyboardInterrupt: 

In [78]:
##################################################
extra work
def extractDomain(x):
    return x.strip().split('@')[1]

email = df['contact_email'].apply(extractDomain)
email.drop_duplicates(inplace=True)
email = email.reset_index(drop=True)
print(email.isnull().sum())
print(len(email))

0
2816


In [79]:
df_backup = df

In [ ]:
import requests

url = "https://api.apollo.io/api/v1/organizations/bulk_enrich"

headers = {
    'Cache-Control': 'no-cache',
    'Content-Type': 'application/json'
}
df_domain = pd.read_csv('industry.csv',header=0)
df_backup = df_domain.copy()
df_domain = df_domain[df_domain['industry'].isnull()]
df_domain = df_domain.reset_index(drop=True)

for i in range(0,len(df_domain),10):
        domain = df_domain["contact_email"][i:i+10].tolist()
        data = {
            "api_key": "ehMfh3qDOoYl174_gXvXeg",
            "domains": domain
        }
        try:
            response = requests.request("POST", url, headers=headers, json=data)
            if bool(response.json()):
                if bool(response.json()['organizations']):
                    org_response = response.json()['organizations']
                    for j in range(0,10):
                        if bool(org_response[j]):
                            industry = org_response[j]["industry"]
                            domain_name = org_response[j]["primary_domain"]
                            if bool(industry):
                                idx_df = df_backup[df_backup['contact_email']== domain_name].index
                                df_backup.iloc[idx_df,1] = industry
                            else:
                                idx_df = df_backup[df_backup['contact_email']== domain_name].index
                                df_backup.iloc[idx_df,1] = "No Industry Information"
        except IOError as e:
        # Handle a more general exception (if any other unexpected exception occurs)
            errormsg = str(e)
            print(f"Unexpected error: {errormsg}")





In [ ]:


df_backup[df_backup['contact_email']=='steckel.com'].index

df_domain



In [ ]:
df_backup[df_backup["industry"].notnull()]

In [ ]:
df_domain = pd.read_csv('industry.csv',header=0)
df_backup = df_domain.copy()

df_backup[df_backup["industry"].notnull()]